# <center> Логические алгоритмы классификации</center>

## Определение информативности 

Предикат $φ$ тем более информативен, чем больше он выделяет объектов «своего» класса $c \in Y$ по сравнению с объектами всех остальных «чужих» классов. Свои объекты называют также позитивными (positive), а чужие — негативными (negative). Введём следующие обозначения:
- $P_c$ — число объектов класса $c$ в выборке $X_l$;
- $p_c(φ)$ — из них число объектов, для которых выполняется условие $φ(x) = 1$; 
- $N_c$ — число объектов всех остальных классов $Y \backslash \{c\}$ в выборке $X_l$;
- $n_c(φ)$ — из них число объектов, для которых выполняется условие $φ(x) = 1$.

Задача построения информативного предиката $φ$ сводится к оптимизации по двум критериям:
- $p_c(φ) \rightarrow max$
- $n_c(φ) \rightarrow min$

Предикат называется логической $\varepsilon, \delta $ закономерностью для класса $c\in Y$, если $E_c(φ, X^l) \leq \varepsilon$ и $D_c(φ, X^l) \geq \delta$, при заданных достаточно малом  $\varepsilon$ и достаточно большом  $\delta $ из отрезка $[0, 1]$


$E_c(φ, X^l) = \frac{n_c(φ)}{p_c(φ)+n_c(φ)},  D_c(φ, X^l) = \frac{p_c(φ)}{l}$

- $E_c$-- доля негативных среди всех выделяемых объектов;
-  $D_c$-- доля выделяемых позитивных объектов.

### Статистическое определение информативности

Информативность предиката $φ(x)$ относительно класса $c\in Y$ по выборке $X^l$ есть: 

$$I_c(φ, X^l) = - \ln h_{P_c, N_c} (p_c(φ), n_c(φ))$$

Предикат $φ(x)$ будем называть статистической закономерностью для класса $c$, если $I_c(φ, X^l) \leq I_0$ при заданном достаточно большом $I_0$.

$h_{P,N}(p, n) = \frac{C_P^p *C_N^n}{C_{P+N}^{p+n}}$

In [14]:
from scipy.special import binom

In [16]:
def I_c(p, n, P, N):
    return - np.log((binom(P, p) * binom (N, n)/ binom(P+N, p+n)))

In [86]:
I_c(200, 0, 200, 100)

187.9344871970993

### Энтропийное определение информативности

Если имеются два исхода $ω_0, ω_1$ с вероятностями $q_0$ и $q_1 = 1 − q_0$, то количество информации, связанное с исходом $ω_i$, по определению равно − $log_2 q_i$. Энтропия определяется как матожидание количества информации:

$$H(q_0, q_1) = −q_0 log_2 q_0 − q_1 log_2 q_1$$

Предикат $ϕ$ выделил $p$ объектов из $P$,принадлежащих классу $c$, и $n$ объектов из $N$, не принадлежащих классу $c$.
Тогда энтропия выборки $\{x∈X_ℓ: ϕ(x)=1\}$ есть $H(p,n)$. Вероятность появления объекта из этой выборки оценивается как $ \frac{p+n}{P+N}$. 

Энтропия всей выборки после получения информации равна:

$H_ϕ(P,N,p,n) = \frac {p+n}{P+N} H(p,n) + \frac {P +N −p−n} {P+N} H(P-p, N-n)$

Уменьшение энтропии составляет:

$IGain_с(ϕ,X^ℓ) = H(P,N) − H_ϕ(P,N,p,n)$.

In [19]:
import numpy as np

In [53]:
def H(p):
    if p==0 or p==1:
        return 0
    else:
        return -p*np.log2(p)-(1-p)*np.log2(1-p)

In [54]:
def H_1(P, N, p, n): 
    l = P+N
    return (p+n)/l* H(p/(p+n)) + (l-p-n)/l*H((P-p)/(l-p-n))

In [56]:
(H(P/(P+N)) - H_1(200, 100, 50, 0))*300

32.7511016026797

### Критерий Джинни

Если представим $h(p)$, как $4q(1-q)$, то получим скорректированный $IGain_c$

In [62]:
def G(p):
    return 4*p*(1-p)

In [63]:
def H_2(P, N, p, n): 
    l = P+N
    return (p+n)/l* G(p/(p+n)) + (l-p-n)/l*G((P-p)/(l-p-n))

In [64]:
(G(P/(P+N)) - H_2(200, 100, 50, 0))*300

26.66666666666667

In [71]:
import plotly.express as px
import pandas as pd

In [79]:
p = np.arange(0,1.01,0.01)
n = [H(i) for i in p]
g = [G(i) for i in p]

In [80]:
df = pd.DataFrame({'Вероятность':p, 'Энтропийный': n, 'Джинни': g})

In [81]:
px.line(df, x = 'Вероятность', y = ['Энтропийный','Джинни'])

## Методы поиска информативных закономерностей

### Бинаризация количественных признаков

Произвольный признак $f: X → D_f$ порождает семейство предикатов, проверяющих попадание значения $f(x)$ в определённые подмножества множества $D_f$.

- Если $f$ номинальный признак: 
    - $β(x) = [f(x) = d], d ∈ D_f$;
    - $β(x) = [f(x) ∈ D′], D′ ⊂ D_f.$
- Если $f$ порядковый или количественный признак: 
    - $β(x) = [f(x) \leq d], d ∈ D_f$; 
    - $β(x) = [d \leq f(x) \leq d′ ], \quad d,d′∈D_f, d<d′$.

В случае количественных признаков $f: X → R$ имеет смысл брать только та кие значения порогов $d$, которые поразному разделяют выборку $X^ℓ$.

$$d_i = \frac {f^{(i)} +f^{(i+1)}}{2},   \quad  f^{(i)} \neq f^{(i+1)}, \quad  i = 1,...,ℓ − 1$$

где $f(1) \leq ... \leq f(ℓ)$ ,  последовательность значений признака $f$ на объектах выборки $f(x_1),...,f(x_ℓ)$, упорядоченная по возрастанию.

**Вход**: 
- $f(x)$ признак; 
- $c ∈ Y$ выделенный класс; 
- $X_l ={x_i,y_i}_{i=1}^l$ выборка, упорядоченная по возрастанию $f(x_i)$; 
- $r$ желаемое количество зон; 
- $δ_0$ порог слияния зон (по умолчанию $δ_0 = 0$).

**Выход**: $D=\{d_1,...,d_r\}$ строго возрастающая последовательность порогов; 



1. D := ∅; 
2. для всех i = 2,...,l 
3. если $f(x_{i−1}) \neq f(x_i)$ и $[y_{i−1} = c] \neq [y_i = c]$, то добавить новый порог $\frac{1}{2}(f(x_{i−1}) + f(x_i))$ в конец последовательности $D$; 
4. повторять
    - для всех $d_i ∈ D, i =1,...,|D|−1$ 
    - вычислить выигрыш от слияния тройки соседних зон $ζ_{i−1}, ζ_i, ζ_{i+1}$: 
        - $δI_i := I_c(ζ_{i−1} ∨ ζ_i ∨ ζ_{i+1}) − max\{I_c(ζ{i−1}),I_c(ζ_i),I_c(ζ_{i+1})\}$; 
    - найти тройку зон, для которой слияние наиболее выгодно: 
        - $i := arg  \underset{s}{max} \delta I_s$; 
    - если $δI_i > δ_0$ то слить зоны $ζ_{i−1}, ζ_i, ζ_{i+1}$ удалив пороги $d_i$ и $d_{i+1}$ из последовательности $D$;  
    
5. пока $|D| > r +1$.

In [67]:
def I_c(y):
    
    if len(y) == 0:
        return 0
    probabilities = np.bincount(y) / len(y)
    return -np.sum(probabilities * np.log2(probabilities + 1e-9))  #добавление маленького числа для предотвращения логарифма нуля

In [68]:
def binarization(f, c, X, r, delta_0=0):
    D = []
    l = len(X)

    #находим все возможные пороги
    for i in range(1, l - 1):
        if f[X[i - 1][0]] != f[X[i][0]] and (X[i - 1][1] == c) != (X[i][1] == c):
            new_threshold = 0.5 * (f[X[i - 1][0]] + f[X[i][0]])
            D.append(new_threshold)

    D = sorted(set(D))  

    #сокращение порогов
    while len(D) > r + 1:
        delta_I = []

        for i in range(1, len(D) - 1):
            zone1 = [X[j] for j in range(len(X)) if f[X[j][0]] < D[i]]
            zone2 = [X[j] for j in range(len(X)) if D[i] <= f[X[j][0]] < D[i + 1]]
            zone3 = [X[j] for j in range(len(X)) if f[X[j][0]] >= D[i + 1]]

            #вычисляем информационную приросту для каждой зоны
            I1 = I_c(np.array([x[1] for x in zone1])) if len(zone1) > 0 else 0
            I2 = I_c(np.array([x[1] for x in zone2])) if len(zone2) > 0 else 0
            I3 = I_c(np.array([x[1] for x in zone3])) if len(zone3) > 0 else 0

            #общая энтропия
            combined_I = I_c(np.array([x[1] for x in zone1 + zone2 + zone3]))
            delta_I.append(combined_I - max(I1, I2, I3))

        i_max = np.argmax(delta_I)

        if delta_I[i_max] > delta_0:
            D.pop(i_max)
        else:
            break  #если прирост не превышает delta_0, выходим из цикла

    return D

In [69]:
f = {0: 1.5, 1: 2.3, 2: 3.1, 3: 4.8, 4: 6.0}  
c = 1 
X = [[0, 0], [1, 1], [2, 0], [3, 1], [4, 0]]  
r = 2  

thresholds = binarization(f, c, X, r)
print("Пороги:", thresholds)

Пороги: [1.9, 2.7, 3.95]


### Жадный алгоритм построения решающего списка

Решающий список это алгоритм классификации $a:X→Y$, который задаётся набором закономерностей $φ_1(x),...,φ_T(x)$, приписанных к классам $c_1,...,c_T ∈Y$ соответственно.

**Вход**: 
- $X^l$ обучающая выборка; 
- $Φ$ семейство предикатов, из которого выбираются закономерности; 
- $T_{max}$ максимальное допустимое число правил в списке; 
- $I_{min}$ минимальная допустимая информативность правил в списке; 
- $E_{max}$ максимальная допустимая доля ошибок на обучающей выборке; 
- $l_0$ максимальное допустимое число отказов; 

**Выход**: решающий список $\{φ_t,c_t\}^T_{t=1}$; 

**Алгоритм**:

1.  $U := X^l$
2. для всех $t := 1,...,T_{max}$ 
- $c := c_t$ выбрать класс из $Y$, для которого будет строиться правило; 
- найти наиболее информативное правило при ограничении на долю ошибок: $φ_t :=  arg  \underset{ φ∈Φ′}{max} I_c(φ,U)$, где $Φ′ = \{φ ∈ Φ: E_c(φ,U)\leq E_{max}\}$ ; 
- если $I_c(φ_t,U) < I_{min}$ то выход; 
- исключить из выборки объекты, выделенные правилом $φ_t$: 
$U:=\{x ∈ U :φ_t(x) =0\}$; 
-если $|U|\leq l_0$ то выход;

In [70]:
def decision_list_alg(data, F, T_max, I_min, E_max, l_0):
    rules = [] #решающий список
    U = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    cs = y.unique() #список классов
    
    for t in range(T_max):
        c = np.random.choice(cs) #выбираем случайный класс
        F_c = F[c]               #отбираем набор предикатов для выбранного класса
        best_dI = -np.inf
        best_f = -1
        P, N = (y == c).sum(), (y != c).sum() #число объектов класса "c" и не "c"
        
        for feature, dl, dr in F_c:
            if isinstance(feature, int):
                rule = (U.iloc[:, feature] > dl) & (U.iloc[:, feature] <= dr)
            else:
                rule = (U[feature] > dl) & (U[feature] <= dr)
            y_rule = y[rule]
            if E_c(y_rule, c) > E_max or ((feature, dl, dr), c) in rules: #если слишком много ошибок, не рассматриваем
                continue
            I = Igain(P, N, y_rule, c)
            if I >= best_dI:
                best_dI = I
                best_f = (feature, dl, dr)
        
        if best_dI < I_min:
            return rules
        
        feature = best_f[0]
        if isinstance(feature, int):
            drop_candidates = (U.iloc[:, feature] > dl) & (U.iloc[:, feature] <= dr)
        else:
            drop_candidates = (U[feature] > dl) & (U[feature] <= dr)
        U = U[~drop_candidates]
        y = y[~drop_candidates]
        
        rules.append((best_f, c))
        
        if U.shape[0] <= l_0:
            return rules
    return rules

### Алгоритм построения решающего дерева

*Деревом* называетс конечный связный ациклический граф с множеством вершин $V$, имеющий выделенную вершину $v_0 \in V$, в которую не входит ни одно ребро. Эта вершина называется *корнем* графа. Вершины, не имеющие выходящих ребер называются *терминальными* или *листьями*. Остальные вершины называются *внутренними*. Дерево называется *бинарным*, если из любой его внутренней вершины выходит ровно два ребра.

*Бинарное решающее дерево* -- алгоритм классификации, задающийся бинарным деревом, в котором каждой внутренней вершине $v \in V$ приписан предикат $\beta_v: X \rightarrow \{0,1\}$, каждой терминальной вершине $v \in V$ приписанов имя класса $c_v\in Y$

**Алгоритм построение решающего дерева ID3**

**Вход:** 
- $U$ - обучающая выборка;
- $\mathcal{B}$ - множество элементарных предикатов.

**Выход**: 

**Процедура LearnID3(U)**:
1. Если все объекты из $U$ лежат в одном классе $c \in Y$, то: 
    - создать новый лист $v$; 
    - $c_v$:=c
    - вернуть ($v$)
2. Иначе:
    1. Найти предикат с максимальной информативностью: $\beta :=  arg  \underset{ \beta \in \mathcal{B}}{max} I(\beta, U)$
    2. Разбить выборку на две части $U=U_0 \cup \ U_1$ по предикату $\beta$:
        - $U_0:=\{x\in U:\beta(x) = 0\}$
        - $U_1:=\{x\in U:\beta(x) = 1\}$
    3. Если $U_0 = \emptyset$ или $U_1 = \emptyset$:
        - создать новый лист $v$;
        - $c_v$ - класс, в котором находится большинство объектов из $U$.
    4. Иначе: 
        - создать внутреннюю вершину $v$;
        - $\beta_v = \beta$;
        - $L_v = LearnID3(U_0)$;
        - $R_v = LearnID3(U_1)$;
        
    5.вернуть  ($v$)
    

In [71]:
def decision_tree(X, f, r, delta_0, depth=0):
    #если все метки одинаковы
    if len(set(x[1] for x in X)) == 1:
        return X[0][1]  

    classes = list(set(x[1] for x in X))
    
    if len(classes) == 1 or depth >= r:
        return max(set(x[1] for x in X), key=lambda x: list(x[1] for x in X).count(x))  #возвращаем наиболее частую метку

    thresholds = binarization(f, classes[0], X, r, delta_0)

    #если пороги не найдены, возвращаем наиболее частую метку
    if not thresholds:
        return max(set(x[1] for x in X), key=lambda x: list(x[1] for x in X).count(x))

    # рекурсивно строим поддеревья
    tree = {}
    for threshold in thresholds:
        left_split = [x for x in X if f[x[0]] < threshold]
        right_split = [x for x in X if f[x[0]] >= threshold]

        tree[threshold] = {
            'left': decision_tree(left_split, f, r, delta_0, depth + 1),
            'right': decision_tree(right_split, f, r, delta_0, depth + 1)
        }

    return tree

In [72]:
X = [(0, 0), (1, 1), (2, 0), (3, 1), (4, 0)]
f = [1.0, 2.0, 1.5, 2.5, 1.8]
r = 2
delta_0 = 0.01 

tree = decision_tree(X, f, r, delta_0)
print(tree)

{1.5: {'left': 0, 'right': {1.75: {'left': 0, 'right': 1}, 2.0: {'left': 0, 'right': 1}}}, 1.75: {'left': 0, 'right': 1}, 2.0: {'left': 0, 'right': 1}}
